In [2]:
import numpy as np

## Вспомогательные функции

In [3]:
def print_array(matrix, msg="", sep='\n'):
    n = matrix.shape[0]
    m = None

    if msg != "":
        print(msg)
    if len(matrix.shape) == 2:
        m = matrix.shape[1]

    if len(matrix.shape) == 2:
        for i in range(n):
            for j in range(m):
                print('{:>8.5f}'.format(matrix[i, j]), end=' ')
            print(sep, end='')

    elif len(matrix.shape) == 1:
        for i in range(n):
            print('{:>8.5f}'.format(matrix[i]), end=' ')
        print(sep, end='')

In [4]:
def symmetric_check(matrix, eps):
    for i in range(1, len(matrix)):
        for j in range(i):
            if (abs(matrix[i][j] - matrix[j][i]) >= eps):
                raise ValueError("Матрица должна быть симметричной!")

## Вспомогательные функции для метода вращений

In [5]:
def check_equal_dim(matrix):
     if matrix.shape[0] != matrix.shape[1]:
        raise ValueError("Матрица размера не NxN")

In [6]:
def calc_non_diag(matrix):
    check_equal_dim(matrix)

    n = matrix.shape[0]
    sum_n_diag = 0
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            sum_n_diag += abs(matrix[i, j])
    return sum_n_diag

In [7]:
def max_no_diag(matrix):
    check_equal_dim(matrix)

    n = matrix.shape[0]
    max_val = matrix[0, 1]
    val_row = 0
    val_col = 1
    for i in range(n):
        for j in range(i + 1, n):
            if abs(matrix[i, j]) > abs(max_val):
                max_val = matrix[i, j]
                val_row = i
                val_col = j

    return max_val, val_row, val_col

## Метод вращений

In [8]:
def find_eigen(matrix, err):
    check_equal_dim(matrix)
    symmetric_check(matrix, err)
    A = matrix.copy()
    n = A.shape[0]
    rotate_matrix = np.eye(n)  # diagonal matrix
    eig_vec = np.zeros(shape=A.shape)
    iteration = 0
    while calc_non_diag(A) > err:
        
        max_el, p, q = max_no_diag(A)

        if A[p, p] == A[q, q]:
            if max_el > 0:
                teta = np.pi / 4
            else:
                teta = -1 * np.pi / 4
        else:
            teta = np.arctan((2 * max_el) / (A[p, p] - A[q, q])) / 2
    
        # fill rotate matrix
        rotate_matrix = np.eye(n)
        rotate_matrix[p, p] = np.cos(teta)
        rotate_matrix[q, q] = np.cos(teta)
        rotate_matrix[p, q] = np.sin(teta) * -1
        rotate_matrix[q, p] = np.sin(teta)

        A = rotate_matrix.T @ A @ rotate_matrix

        if iteration != 0:
            eig_vec = eig_vec @ rotate_matrix
        else:
            eig_vec = rotate_matrix.copy()

        iteration += 1

    return A, eig_vec, iteration

In [14]:
def solve(A, E):
    try:
        eig_val, eig_vec, steps = find_eigen(A, E)
        print_array(A, 'Матрица согласно условию:')
        print("\nВЫПОЛНЯЮТСЯ ВЫЧИСЛЕНИЯ МЕТОДОМ ВРАЩЕНИЙ...")
        print_array(eig_val, '\nМатрица собственных значений:')
        w, v = np.linalg.eig(A)
        print_array(w, 'Собственные значения:')
        print_array(v, 'Собственные вектора:')
        print("Количество итераций:", steps)
        print("\nПроверка с помощью numpy:\n")
        
        print_array(w, 'Собственные значения (numpy):')
        print_array(v, 'Собственные вектора (numpy):')
    except ValueError as err:
        print(err)
        print("Проводится решение другим алгоритмом...")
        w, v = np.linalg.eig(A)
        print_array(w, 'Собственные значения:')
        print_array(v, 'Собственные вектора:')

## Тестовый пример 1

In [15]:
E = 0.000001
A = np.array([
    [5, 1, 2],
    [1, 4, 1],
    [2, 1, 3]
])

solve(A, E)

Матрица согласно условию:
 5.00000  1.00000  2.00000 
 1.00000  4.00000  1.00000 
 2.00000  1.00000  3.00000 

ВЫПОЛНЯЮТСЯ ВЫЧИСЛЕНИЯ МЕТОДОМ ВРАЩЕНИЙ...

Матрица собственных значений:
 6.89511  0.00000  0.00000 
 0.00000  3.39730  0.00000 
-0.00000  0.00000  1.70760 
Собственные значения:
 6.89511  3.39730  1.70760 
Собственные вектора:
-0.75258 -0.45794 -0.47320 
-0.43170  0.88574 -0.17060 
-0.49725 -0.07589  0.86428 
Количество итераций: 7

Проверка с помощью numpy:

Собственные значения (numpy):
 6.89511  3.39730  1.70760 
Собственные вектора (numpy):
-0.75258 -0.45794 -0.47320 
-0.43170  0.88574 -0.17060 
-0.49725 -0.07589  0.86428 


## Тестовый пример 2

In [16]:
E = 0.000001
A = np.array([
    [1, -3, 2],
    [-3, 0, 1],
    [2, 1, 1]
])

solve(A, E)

Матрица согласно условию:
 1.00000 -3.00000  2.00000 
-3.00000  0.00000  1.00000 
 2.00000  1.00000  1.00000 

ВЫПОЛНЯЮТСЯ ВЫЧИСЛЕНИЯ МЕТОДОМ ВРАЩЕНИЙ...

Матрица собственных значений:
 3.88824  0.00000  0.00000 
-0.00000 -3.50331  0.00000 
 0.00000  0.00000  1.61507 
Собственные значения:
-3.50331  3.88824  1.61507 
Собственные вектора:
-0.62496 -0.78029 -0.02384 
-0.65598  0.50834  0.55792 
 0.42322 -0.36432  0.82955 
Количество итераций: 7

Проверка с помощью numpy:

Собственные значения (numpy):
-3.50331  3.88824  1.61507 
Собственные вектора (numpy):
-0.62496 -0.78029 -0.02384 
-0.65598  0.50834  0.55792 
 0.42322 -0.36432  0.82955 


## Тестовый пример 3

In [17]:
E = 0.000001
A = np.array([
    [1, 1, 1, 0],
    [1, 1, 0, 1],
    [1, 0, 1, 1],
    [0, 1, 1, 1]
])

solve(A, E)

Матрица согласно условию:
 1.00000  1.00000  1.00000  0.00000 
 1.00000  1.00000  0.00000  1.00000 
 1.00000  0.00000  1.00000  1.00000 
 0.00000  1.00000  1.00000  1.00000 

ВЫПОЛНЯЮТСЯ ВЫЧИСЛЕНИЯ МЕТОДОМ ВРАЩЕНИЙ...

Матрица собственных значений:
 3.00000  0.00000  0.00000  0.00000 
 0.00000  1.00000  0.00000  0.00000 
 0.00000  0.00000  1.00000  0.00000 
 0.00000  0.00000  0.00000 -1.00000 
Собственные значения:
-1.00000  1.00000  3.00000  1.00000 
Собственные вектора:
 0.50000 -0.70711 -0.50000 -0.01305 
-0.50000 -0.00000 -0.50000  0.70699 
-0.50000  0.00000 -0.50000 -0.70699 
 0.50000  0.70711 -0.50000  0.01305 
Количество итераций: 4

Проверка с помощью numpy:

Собственные значения (numpy):
-1.00000  1.00000  3.00000  1.00000 
Собственные вектора (numpy):
 0.50000 -0.70711 -0.50000 -0.01305 
-0.50000 -0.00000 -0.50000  0.70699 
-0.50000  0.00000 -0.50000 -0.70699 
 0.50000  0.70711 -0.50000  0.01305 




## Тестовый пример 4

In [18]:
E = 0.000001
A = np.array([
    [11, 11, 1, 0],
    [1, 1, 0, 1],
    [1, 0, 1, 1],
    [0, 1, 1, 1]
])

solve(A, E)

Матрица должна быть симметричной!
Проводится решение другим алгоритмом...
Собственные значения:
12.09902 -1.00000  1.90098  1.00000 
Собственные вектора:
-0.99158 -0.50000 -0.44798 -0.50000 
-0.09081  0.50000  0.33968  0.50000 
-0.09081  0.50000  0.33968 -0.50000 
-0.01636 -0.50000  0.75402  0.50000 


## Решение задания

In [13]:
E = 0.000001

k = 3

C = np.array([
    [0.2, 0, 0.2, 0, 0],
    [0, 0.2, 0, 0.2, 0],
    [0.2, 0, 0.2, 0, 0.2],
    [0, 0.2, 0, 0.2, 0],
    [0, 0, 0.2, 0, 0.2]
])

D = np.array([
    [2.33, 0.81, 0.67, 0.92, -0.53],
    [0.81, 2.33, 0.81, 0.67, 0.92],
    [0.67, 0.81, 2.33, 0.81, 0.92],
    [0.92, 0.67, 0.81, 2.33, -0.53],
    [-0.53, 0.92, 0.92, -0.53, 2.33]
])

A = k * C + D

solve(A, E)

Матрица согласно условию:
 2.93000  0.81000  1.27000  0.92000 -0.53000 
 0.81000  2.93000  0.81000  1.27000  0.92000 
 1.27000  0.81000  2.93000  0.81000  1.52000 
 0.92000  1.27000  0.81000  2.93000 -0.53000 
-0.53000  0.92000  1.52000 -0.53000  2.93000 

ВЫПОЛНЯЮТСЯ ВЫЧИСЛЕНИЯ МЕТОДОМ ВРАЩЕНИЙ...

Матрица собственных значений:
 2.50483 -0.00000 -0.00000  0.00000  0.00000 
-0.00000  6.06293  0.00000  0.00000 -0.00000 
-0.00000  0.00000  4.08408  0.00000  0.00000 
 0.00000  0.00000  0.00000  1.60465  0.00000 
 0.00000 -0.00000  0.00000 -0.00000  0.39352 
Собственные значения:
 2.50483  6.06293  4.08408  1.60465  0.39352 
Собственные вектора:
 0.56083  0.43337 -0.39300 -0.44298  0.38339 
-0.58447  0.50629  0.02368 -0.54518 -0.32295 
 0.43481  0.54770  0.26768  0.37704 -0.54512 
-0.38669  0.42867 -0.44712  0.60357  0.32017 
-0.07269  0.26857  0.75725  0.01025  0.59081 
Количество итераций: 28

Проверка с помощью numpy:

Собственные значения (numpy):
 6.06293  4.08408  0.39352  2.50483  1